## Observation

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from copy import deepcopy as cp
import gc; gc.enable()
import matplotlib.pyplot as plt
pd.options.display.max_columns = 100
%matplotlib inline

In [2]:
col_info = {
    'high_card_cat_cols': [],
    'cat_cols':[],
    'num_cols': [], # numerical columns to be boxcox + norm to [0,1]
    'other_cols': []
}

In [3]:
train_df = pd.read_csv("data/train.csv", parse_dates = ["activation_date"])
test_df = pd.read_csv("data/test.csv", parse_dates = ["activation_date"])
print("Train file rows and columns are : ", train_df.shape)
print("Test file rows and columns are : ", test_df.shape)

Train file rows and columns are :  (1503424, 18)
Test file rows and columns are :  (508438, 17)


In [4]:
train_df = train_df.sort_values('activation_date').reset_index(drop=True)

In [5]:
train_y = train_df[['deal_probability']]
train_y.to_csv('regression_target.csv', index=False)

train_y.loc[:, 'deal_probability'] = (train_y['deal_probability'].values > .5).astype(int)
train_y.to_csv('classification_target.csv', index=False)

C:\Users\khyeh\Anaconda3\lib\site-packages\pandas\core\indexing.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [6]:
train_df.head(3)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,3b896605f03c,9457abfa0ebf,Самарская область,Самара,Личные вещи,Детская одежда и обувь,Для мальчиков,Верхняя одежда,86-92 см (1-2 года),Зимний комбенизон,Зимний комбенизон для малыша,500.0,12,2017-03-15,Private,a805aa774699aca42e538e1626ef70f806dd38d1bf85eb...,1008.0,0.0
1,c81c34d3aac5,0b161da2b144,Иркутская область,Братск,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,NaN,Чехол для айфона 6s,Твёрдый пластик,100.0,5,2017-03-15,Private,76c39c52a894c51499da3ee2ddecd98da533ccbdae5096...,2909.0,0.0
2,a8ab6225dd06,93d527a1ca66,Пермский край,Краснокамск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Верхняя одежда,44–46 (M),Продам кожанную куртку,NaN,1500.0,32,2017-03-15,Private,d7cb3c4a84ce9f95958f2995474140b28ae7ad1cd0d6de...,527.0,0.0


In [7]:
test_df['deal_probability'] = np.zeros((test_df.shape[0],))
all_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

In [8]:
display(train_df.head(3))
display(test_df.head(3))

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,3b896605f03c,9457abfa0ebf,Самарская область,Самара,Личные вещи,Детская одежда и обувь,Для мальчиков,Верхняя одежда,86-92 см (1-2 года),Зимний комбенизон,Зимний комбенизон для малыша,500.0,12,2017-03-15,Private,a805aa774699aca42e538e1626ef70f806dd38d1bf85eb...,1008.0,0.0
1,c81c34d3aac5,0b161da2b144,Иркутская область,Братск,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,NaN,Чехол для айфона 6s,Твёрдый пластик,100.0,5,2017-03-15,Private,76c39c52a894c51499da3ee2ddecd98da533ccbdae5096...,2909.0,0.0
2,a8ab6225dd06,93d527a1ca66,Пермский край,Краснокамск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Верхняя одежда,44–46 (M),Продам кожанную куртку,NaN,1500.0,32,2017-03-15,Private,d7cb3c4a84ce9f95958f2995474140b28ae7ad1cd0d6de...,527.0,0.0


,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,На ангарском,NaN,66,2017-04-18,Private,a8b57acb5ab304f9c331ac7a074219aed4d349d8aef386...,2020.0,0.0
1,65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,NaN,NaN,Продам велосипед,"Продам велосипед KAMA F200,в нормальном состо...",3000.0,4,2017-04-16,Private,NaN,NaN,0.0
2,8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,NaN,NaN,BBK,Продам новый телевизор BBK 32 диагональ смарт...,15000.0,15,2017-04-17,Private,8c361112cb049745ef2d1b0ae73594fc5c107286b0c942...,2960.0,0.0


### Merge period features

In [9]:
period_df = pd.read_csv('ad_period_features.csv')

C:\Users\khyeh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
all_df.head(3)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,3b896605f03c,9457abfa0ebf,Самарская область,Самара,Личные вещи,Детская одежда и обувь,Для мальчиков,Верхняя одежда,86-92 см (1-2 года),Зимний комбенизон,Зимний комбенизон для малыша,500.0,12,2017-03-15,Private,a805aa774699aca42e538e1626ef70f806dd38d1bf85eb...,1008.0,0.0
1,c81c34d3aac5,0b161da2b144,Иркутская область,Братск,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,NaN,Чехол для айфона 6s,Твёрдый пластик,100.0,5,2017-03-15,Private,76c39c52a894c51499da3ee2ddecd98da533ccbdae5096...,2909.0,0.0
2,a8ab6225dd06,93d527a1ca66,Пермский край,Краснокамск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Верхняя одежда,44–46 (M),Продам кожанную куртку,NaN,1500.0,32,2017-03-15,Private,d7cb3c4a84ce9f95958f2995474140b28ae7ad1cd0d6de...,527.0,0.0


In [11]:
period_df.head(5)

,new_index,item_id,activation_date,date_from,date_to,duration_1,duration_2,duration_3,renewed_count,is_renewed
0,0fceba64da0d 2017-01-01,0fceba64da0d,2017-01-01,2017-03-15,2017-03-17,73.0,75.0,2,0,0
1,e06329e1ce56 2017-01-01,e06329e1ce56,2017-01-01,2017-03-15,2017-03-17,73.0,75.0,2,0,0
2,0d3f55aa2786 2017-01-01,0d3f55aa2786,2017-01-01,2017-04-12,2017-04-18,101.0,107.0,6,0,0
3,02530587479d 2017-01-01,02530587479d,2017-01-01,2017-04-12,2017-04-18,101.0,107.0,6,0,0
4,a2971556d92b 2017-01-01,a2971556d92b,2017-01-01,2017-03-15,2017-03-16,73.0,74.0,1,0,0


In [12]:
pd.Series(all_df.activation_date.isin(period_df.activation_date.values)).value_counts()

True     2011797
False         65
Name: activation_date, dtype: int64

In [13]:
gp = period_df.groupby('activation_date')

In [14]:
all_df.loc[:,'same_activate_cnt'] = all_df['activation_date'].astype(str).map(gp['duration_1'].count())

In [15]:
# duration_1 duration_2 duration_3 renewed_count is_renewed
mapped_cols = ['duration_1', 'duration_2', 'duration_3', 'renewed_count', 'is_renewed']
new_cols = ['same_activate_cnt']
for col in mapped_cols:
    print('Processing ', col)
    all_df.loc[:,col+'_mean'] = all_df['activation_date'].astype(str).map(gp[col].mean())
    all_df.loc[:,col+'_med'] = all_df['activation_date'].astype(str).map(gp[col].median())
    
    if col != 'is_renewed':
        all_df.loc[:,col+'_min'] = all_df['activation_date'].astype(str).map(gp[col].min())
        all_df.loc[:,col+'_max'] = all_df['activation_date'].astype(str).map(gp[col].max())
    all_df.loc[:,col+'_std'] = all_df['activation_date'].astype(str).map(gp[col].std())
    new_cols.extend([col+'_mean',col+'_med',col+'_min',col+'_max',col+'_std'])

Processing  duration_1
Processing  duration_2
Processing  duration_3
Processing  renewed_count
Processing  is_renewed


In [16]:
new_cols.remove('is_renewed_min')
new_cols.remove('is_renewed_max')
print(new_cols)
all_df[new_cols].head(3)

['same_activate_cnt', 'duration_1_mean', 'duration_1_med', 'duration_1_min', 'duration_1_max', 'duration_1_std', 'duration_2_mean', 'duration_2_med', 'duration_2_min', 'duration_2_max', 'duration_2_std', 'duration_3_mean', 'duration_3_med', 'duration_3_min', 'duration_3_max', 'duration_3_std', 'renewed_count_mean', 'renewed_count_med', 'renewed_count_min', 'renewed_count_max', 'renewed_count_std', 'is_renewed_mean', 'is_renewed_med', 'is_renewed_std']


,same_activate_cnt,duration_1_mean,duration_1_med,duration_1_min,duration_1_max,duration_1_std,duration_2_mean,duration_2_med,duration_2_min,duration_2_max,duration_2_std,duration_3_mean,duration_3_med,duration_3_min,duration_3_max,duration_3_std,renewed_count_mean,renewed_count_med,renewed_count_min,renewed_count_max,renewed_count_std,is_renewed_mean,is_renewed_med,is_renewed_std
0,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023
1,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023
2,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023


In [17]:
all_df[new_cols].isnull().sum()

same_activate_cnt     65
duration_1_mean       65
duration_1_med        65
duration_1_min        65
duration_1_max        65
duration_1_std        65
duration_2_mean       65
duration_2_med        65
duration_2_min        65
duration_2_max        65
duration_2_std        65
duration_3_mean       65
duration_3_med        65
duration_3_min        65
duration_3_max        65
duration_3_std        65
renewed_count_mean    65
renewed_count_med     65
renewed_count_min     65
renewed_count_max     65
renewed_count_std     65
is_renewed_mean       65
is_renewed_med        65
is_renewed_std        65
dtype: int64

In [18]:
all_df.loc[:, new_cols] = all_df.loc[:, new_cols].fillna(0)

In [19]:
all_df[new_cols].isnull().sum()

same_activate_cnt     0
duration_1_mean       0
duration_1_med        0
duration_1_min        0
duration_1_max        0
duration_1_std        0
duration_2_mean       0
duration_2_med        0
duration_2_min        0
duration_2_max        0
duration_2_std        0
duration_3_mean       0
duration_3_med        0
duration_3_min        0
duration_3_max        0
duration_3_std        0
renewed_count_mean    0
renewed_count_med     0
renewed_count_min     0
renewed_count_max     0
renewed_count_std     0
is_renewed_mean       0
is_renewed_med        0
is_renewed_std        0
dtype: int64

In [20]:
col_info['num_cols'].extend(['same_activate_cnt', 'duration_1_mean', 'duration_1_med', 'duration_1_min', 'duration_1_max', 'duration_1_std', 'duration_2_mean', 'duration_2_med', 'duration_2_min', 'duration_2_max', 'duration_2_std', 'duration_3_mean', 'duration_3_med', 'duration_3_min', 'duration_3_max', 'duration_3_std', 'renewed_count_mean', 'renewed_count_med', 'renewed_count_min', 'renewed_count_max', 'renewed_count_std', 'is_renewed_mean', 'is_renewed_med', 'is_renewed_std'])

In [21]:
del period_df, gp; gc.collect()

194

### Merge aggregated features

In [22]:
agg_features = pd.read_csv('aggregated_features.csv').set_index('user_id')
agg_features.head(3)

,avg_days_up_user,avg_times_up_user,med_days_up_user,med_times_up_user,n_user_items
user_id,,,,,
00000077ff21,12.5,2.0,12.5,2.0,2
000006497719,19.0,2.0,19.0,2.0,1
00000b4d72f6,3.0,1.0,3.0,1.0,1


In [23]:
new_cols = ['avg_days_up_user', 'avg_times_up_user', 'med_days_up_user', 'med_times_up_user', 'n_user_items']
for col in new_cols:
    all_df.loc[:, col] = all_df['user_id'].map(agg_features[col])
    print(col+' processed')

avg_days_up_user processed
avg_times_up_user processed
med_days_up_user processed
med_times_up_user processed
n_user_items processed


In [24]:
all_df.loc[:, new_cols] = all_df[new_cols].fillna(0)
all_df.isnull().sum()

item_id                       0
user_id                       0
region                        0
city                          0
parent_category_name          0
category_name                 0
param_1                   84486
param_2                  887771
param_3                 1168896
title                         0
description              116276
price                    115947
item_seq_number               0
activation_date               0
user_type                     0
image                    155197
image_top_1              155197
deal_probability              0
same_activate_cnt             0
duration_1_mean               0
duration_1_med                0
duration_1_min                0
duration_1_max                0
duration_1_std                0
duration_2_mean               0
duration_2_med                0
duration_2_min                0
duration_2_max                0
duration_2_std                0
duration_3_mean               0
duration_3_med                0
duration

In [25]:
col_info['num_cols'].extend(['avg_days_up_user', 'avg_times_up_user', 'med_days_up_user', 'med_times_up_user', 'n_user_items'])

In [26]:
del agg_features; gc.collect()

80

### Merge active features

In [27]:
active_features = pd.read_csv('active_features.csv').set_index('activation_date')
active_features.head(3)

,price_mean,price_med,price_std,title_len_mean,title_len_med,title_len_std,title_word_cnt_mean,title_word_cnt_med,title_word_cnt_std,title_unique_word_cnt_mean,title_unique_word_cnt_med,title_unique_word_cnt_std,title_unqiue_percent_mean,title_unqiue_percent_std,description_len_mean,description_len_med,description_len_std,description_word_cnt_mean,description_word_cnt_med,description_word_cnt_std,description_unique_word_cnt_mean,description_unique_word_cnt_med,description_unique_word_cnt_std,description_unqiue_percent_mean,description_unqiue_percent_med,description_unqiue_percent_std,avg_types_unique_percent_item_id,avg_types_unique_percent_city,avg_types_unique_percent_region,avg_types_unique_percent_parent_category_name,avg_types_unique_percent_category_name
activation_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-15,0.690736,0.450292,0.875487,0.864900,0.842954,0.997657,0.864585,0.771784,0.966502,0.863109,0.771784,0.971007,0.496627,1.000000,0.740821,0.595368,0.988326,0.721491,0.592194,0.987343,0.730156,0.603744,0.988465,0.930214,1.0,0.691368,0.000000,0.000000,0.000000,0.000000,0.000000
2017-03-16,1.000000,0.484762,1.000000,0.825442,0.784680,0.963073,0.849247,0.771784,0.951882,0.847952,0.771784,0.956814,0.668187,0.952174,0.712810,0.580474,0.945060,0.694337,0.578231,0.940452,0.705322,0.587556,0.947590,0.924242,1.0,0.717187,0.628655,0.520291,0.520499,0.521624,0.520781
2017-03-17,0.552985,0.490756,0.461283,0.823531,0.784680,0.964162,0.849497,0.771784,0.954169,0.848193,0.771784,0.959250,0.653431,0.957772,0.710411,0.580474,0.946216,0.692407,0.578231,0.942526,0.703484,0.587556,0.950533,0.924492,1.0,0.716694,0.665256,0.529178,0.526283,0.526922,0.526838


In [28]:
for col in active_features.columns.tolist():
    all_df.loc[:, col] = all_df['activation_date'].astype(str).map(active_features[col])
    print(col+' processed')

price_mean processed
price_med processed
price_std processed
title_len_mean processed
title_len_med processed
title_len_std processed
title_word_cnt_mean processed
title_word_cnt_med processed
title_word_cnt_std processed
title_unique_word_cnt_mean processed
title_unique_word_cnt_med processed
title_unique_word_cnt_std processed
title_unqiue_percent_mean processed
title_unqiue_percent_std processed
description_len_mean processed
description_len_med processed
description_len_std processed
description_word_cnt_mean processed
description_word_cnt_med processed
description_word_cnt_std processed
description_unique_word_cnt_mean processed
description_unique_word_cnt_med processed
description_unique_word_cnt_std processed
description_unqiue_percent_mean processed
description_unqiue_percent_med processed
description_unqiue_percent_std processed
avg_types_unique_percent_item_id processed
avg_types_unique_percent_city processed
avg_types_unique_percent_region processed
avg_types_unique_percent_

In [29]:
all_df.head(3)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,same_activate_cnt,duration_1_mean,duration_1_med,duration_1_min,duration_1_max,duration_1_std,duration_2_mean,duration_2_med,duration_2_min,duration_2_max,duration_2_std,duration_3_mean,duration_3_med,duration_3_min,duration_3_max,duration_3_std,renewed_count_mean,renewed_count_med,renewed_count_min,renewed_count_max,renewed_count_std,is_renewed_mean,is_renewed_med,is_renewed_std,avg_days_up_user,avg_times_up_user,med_days_up_user,med_times_up_user,n_user_items,price_mean,price_med,price_std,title_len_mean,title_len_med,title_len_std,title_word_cnt_mean,title_word_cnt_med,title_word_cnt_std,title_unique_word_cnt_mean,title_unique_word_cnt_med,title_unique_word_cnt_std,title_unqiue_percent_mean,title_unqiue_percent_std,description_len_mean,description_len_med,description_len_std,description_word_cnt_mean,description_word_cnt_med,description_word_cnt_std,description_unique_word_cnt_mean,description_unique_word_cnt_med,description_unique_word_cnt_std,description_unqiue_percent_mean,description_unqiue_percent_med,description_unqiue_percent_std,avg_types_unique_percent_item_id,avg_types_unique_percent_city,avg_types_unique_percent_region,avg_types_unique_percent_parent_category_name,avg_types_unique_percent_category_name
0,3b896605f03c,9457abfa0ebf,Самарская область,Самара,Личные вещи,Детская одежда и обувь,Для мальчиков,Верхняя одежда,86-92 см (1-2 года),Зимний комбенизон,Зимний комбенизон для малыша,500.0,12,2017-03-15,Private,a805aa774699aca42e538e1626ef70f806dd38d1bf85eb...,1008.0,0.0,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023,6.5,1.5,6.5,1.5,5.0,0.690736,0.450292,0.875487,0.8649,0.842954,0.997657,0.864585,0.771784,0.966502,0.863109,0.771784,0.971007,0.496627,1.0,0.740821,0.595368,0.988326,0.721491,0.592194,0.987343,0.730156,0.603744,0.988465,0.930214,1.0,0.691368,0.0,0.0,0.0,0.0,0.0
1,c81c34d3aac5,0b161da2b144,Иркутская область,Братск,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,NaN,Чехол для айфона 6s,Твёрдый пластик,100.0,5,2017-03-15,Private,76c39c52a894c51499da3ee2ddecd98da533ccbdae5096...,2909.0,0.0,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023,15.0,2.0,15.0,2.0,2.0,0.690736,0.450292,0.875487,0.8649,0.842954,0.997657,0.864585,0.771784,0.966502,0.863109,0.771784,0.971007,0.496627,1.0,0.740821,0.595368,0.988326,0.721491,0.592194,0.987343,0.730156,0.603744,0.988465,0.930214,1.0,0.691368,0.0,0.0,0.0,0.0,0.0
2,a8ab6225dd06,93d527a1ca66,Пермский край,Краснокамск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Верхняя одежда,44–46 (M),Продам кожанную куртку,NaN,1500.0,32,2017-03-15,Private,d7cb3c4a84ce9f95958f2995474140b28ae7ad1cd0d6de...,527.0,0.0,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023,0.0,0.0,0.0,0.0,0.0,0.690736,0.450292,0.875487,0.8649,0.842954,0.997657,0.864585,0.771784,0.966502,0.863109,0.771784,0.971007,0.496627,1.0,0.740821,0.595368,0.988326,0.721491,0.592194,0.987343,0.730156,0.603744,0.988465,0.930214,1.0,0.691368,0.0,0.0,0.0,0.0,0.0


In [30]:
col_info['other_cols'].extend([
    'price_mean',
    'price_med',
    'price_std',
    'title_len_mean',
    'title_len_med',
    'title_len_std',
    'title_word_cnt_mean',
    'title_word_cnt_med',
    'title_word_cnt_std',
    'title_unique_word_cnt_mean',
    'title_unique_word_cnt_med',
    'title_unique_word_cnt_std',
    'title_unqiue_percent_mean',
    'title_unqiue_percent_std',
    'description_len_mean',
    'description_len_med',
    'description_len_std',
    'description_word_cnt_mean',
    'description_word_cnt_med',
    'description_word_cnt_std',
    'description_unique_word_cnt_mean',
    'description_unique_word_cnt_med',
    'description_unique_word_cnt_std',
    'description_unqiue_percent_mean',
    'description_unqiue_percent_med',
    'description_unqiue_percent_std',
    'avg_types_unique_percent_item_id',
    'avg_types_unique_percent_city',
    'avg_types_unique_percent_region',
    'avg_types_unique_percent_parent_category_name',
    'avg_types_unique_percent_category_name'
])

In [31]:
del active_features; gc.collect()

311

In [32]:
all_df.isnull().sum()

item_id                                                0
user_id                                                0
region                                                 0
city                                                   0
parent_category_name                                   0
category_name                                          0
param_1                                            84486
param_2                                           887771
param_3                                          1168896
title                                                  0
description                                       116276
price                                             115947
item_seq_number                                        0
activation_date                                        0
user_type                                              0
image                                             155197
image_top_1                                       155197
deal_probability               

In [33]:
na_cols = ['param_1', 'param_2', 'param_3', 'description']
all_df.loc[:, na_cols] = all_df[na_cols].fillna('unknown')

na_cols =['price', 'image_top_1']
all_df.loc[:, na_cols] = all_df[na_cols].fillna(-1)

In [34]:
all_df.loc[:, 'param_1_is_na'] = (all_df['param_1'] == 'unknown').astype(int)
all_df.loc[:, 'param_2_is_na'] = (all_df['param_2'] == 'unknown').astype(int)
all_df.loc[:, 'param_3_is_na'] = (all_df['param_3'] == 'unknown').astype(int)
all_df.loc[:, 'description_is_na'] = (all_df['description'] == 'unknown').astype(int)
all_df.loc[:, 'price_is_na'] = (all_df['price'] == -1).astype(int)
all_df.loc[:, 'image_top_1_is_na'] = (all_df['image_top_1'] == -1).astype(int)

In [35]:
all_df.drop(['image'], axis=1, inplace=True)

In [36]:
all_df.isnull().sum().max()

0

In [37]:
col_info['other_cols'].extend([
    'param_1_is_na',
    'param_2_is_na',
    'param_3_is_na',
    'description_is_na',
    'price_is_na',
    'image_top_1_is_na'
])

In [38]:
print(col_info)

{'high_card_cat_cols': [], 'cat_cols': [], 'num_cols': ['same_activate_cnt', 'duration_1_mean', 'duration_1_med', 'duration_1_min', 'duration_1_max', 'duration_1_std', 'duration_2_mean', 'duration_2_med', 'duration_2_min', 'duration_2_max', 'duration_2_std', 'duration_3_mean', 'duration_3_med', 'duration_3_min', 'duration_3_max', 'duration_3_std', 'renewed_count_mean', 'renewed_count_med', 'renewed_count_min', 'renewed_count_max', 'renewed_count_std', 'is_renewed_mean', 'is_renewed_med', 'is_renewed_std', 'avg_days_up_user', 'avg_times_up_user', 'med_days_up_user', 'med_times_up_user', 'n_user_items'], 'other_cols': ['price_mean', 'price_med', 'price_std', 'title_len_mean', 'title_len_med', 'title_len_std', 'title_word_cnt_mean', 'title_word_cnt_med', 'title_word_cnt_std', 'title_unique_word_cnt_mean', 'title_unique_word_cnt_med', 'title_unique_word_cnt_std', 'title_unqiue_percent_mean', 'title_unqiue_percent_std', 'description_len_mean', 'description_len_med', 'description_len_std', '

## Text Features (Meta)

In [39]:
def get_contains_feature(df, col1, col2):
    print('processing ', col1, col2)
    res_df = pd.DataFrame()
    str1s = df[col1].values
    str2s = df[col2].values
    
    is_contains = []
    contains_counts = []
    for i in range(len(str1s)):
        str1 = str(str1s[i])
        str2 = str(str2s[i])
        str1 = str1.split(" ")
        str2 = str2.split(" ")
        
        contains_count = 0
        for s in str1:
            if s in str2:
                contains_count += 1
                
        is_contains.append(1 if contains_count > 0 else 0)
        contains_counts.append(contains_count)
        
    res_df['{}_in_{}'.format(col1,col2)] = is_contains
    res_df['{}_in_{}_counts'.format(col1,col2)] = contains_counts
    del is_contains, contains_counts; gc.collect()
    return res_df 

In [40]:
all_df = pd.concat([all_df, get_contains_feature(all_df, 'region', 'title')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'city', 'title')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'parent_category_name', 'title')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'category_name', 'title')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'region', 'description')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'city', 'description')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'parent_category_name', 'description')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'category_name', 'description')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'title', 'description')],  axis=1)

processing  region title
processing  city title
processing  parent_category_name title
processing  category_name title
processing  region description
processing  city description
processing  parent_category_name description
processing  category_name description
processing  title description


In [41]:
all_df.loc[:, 'desc_char_count'] = all_df['description'].map(lambda x: 0 if type(x)==float else len(x))
all_df.loc[:, 'desc_space_count'] = all_df['description'].map(lambda x: 0 if type(x)==float else x.count(' '))
all_df.loc[:, 'desc_unique_words_count'] = all_df['description'].apply(lambda comment: len(set(w for w in comment.split())))
all_df.loc[:, 'desc_percnt_unique_words'] = all_df['desc_unique_words_count'] / (all_df['desc_space_count']+1) * 100. # Count Unique Words

all_df.loc[:, 'surprise_count'] = all_df['description'].map(lambda x: 0 if type(x)==float else x.count('!'))
all_df.loc[:, 'question_count'] = all_df['description'].map(lambda x: 0 if type(x)==float else x.count('?'))
all_df.loc[:, 'quote_count'] = all_df['description'].map(lambda x: 0 if type(x)==float else x.count('\"'))
all_df.loc[:, 'quote_count2'] = all_df['description'].map(lambda x: 0 if type(x)==float else x.count('\''))

all_df.loc[:, 'tit_char_count'] = all_df['title'].map(lambda x: 0 if type(x)==float else len(x))
all_df.loc[:, 'tit_space_count'] = all_df['title'].map(lambda x: 0 if type(x)==float else x.count(' '))
all_df.loc[:, 'tit_unique_words_count'] = all_df['title'].apply(lambda comment: len(set(w for w in comment.split())))
all_df.loc[:, 'tit_percnt_unique_words'] = all_df['tit_unique_words_count'] / (all_df['tit_space_count']+1) * 100. # Count Unique Words

In [42]:
all_df.columns.tolist()

['item_id',
 'user_id',
 'region',
 'city',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'title',
 'description',
 'price',
 'item_seq_number',
 'activation_date',
 'user_type',
 'image_top_1',
 'deal_probability',
 'same_activate_cnt',
 'duration_1_mean',
 'duration_1_med',
 'duration_1_min',
 'duration_1_max',
 'duration_1_std',
 'duration_2_mean',
 'duration_2_med',
 'duration_2_min',
 'duration_2_max',
 'duration_2_std',
 'duration_3_mean',
 'duration_3_med',
 'duration_3_min',
 'duration_3_max',
 'duration_3_std',
 'renewed_count_mean',
 'renewed_count_med',
 'renewed_count_min',
 'renewed_count_max',
 'renewed_count_std',
 'is_renewed_mean',
 'is_renewed_med',
 'is_renewed_std',
 'avg_days_up_user',
 'avg_times_up_user',
 'med_days_up_user',
 'med_times_up_user',
 'n_user_items',
 'price_mean',
 'price_med',
 'price_std',
 'title_len_mean',
 'title_len_med',
 'title_len_std',
 'title_word_cnt_mean',
 'title_word_cnt_med',
 'title_word_cnt_std',


In [43]:
all_df.isnull().sum()

item_id                                       0
user_id                                       0
region                                        0
city                                          0
parent_category_name                          0
category_name                                 0
param_1                                       0
param_2                                       0
param_3                                       0
title                                         0
description                                   0
price                                         0
item_seq_number                               0
activation_date                               0
user_type                                     0
image_top_1                                   0
deal_probability                              0
same_activate_cnt                             0
duration_1_mean                               0
duration_1_med                                0
duration_1_min                          

In [44]:
col_info['num_cols'].extend([
    'region_in_title_counts',
    'city_in_title_counts',
    'parent_category_name_in_title_counts',
    'category_name_in_title_counts',
    'region_in_description_counts',
    'city_in_description_counts',
    'parent_category_name_in_description_counts',
    'category_name_in_description_counts',
    'title_in_description_counts',
    'desc_char_count',
    'desc_space_count',
    'desc_unique_words_count',
    'desc_percnt_unique_words',
    'surprise_count',
    'question_count',
    'quote_count',
    'quote_count2',
    'tit_char_count',
    'tit_space_count',
    'tit_unique_words_count',
    'tit_percnt_unique_words'
])
col_info['other_cols'].extend([
    'region_in_title',
    'city_in_title',
    'parent_category_name_in_title',
    'category_name_in_title',
    'region_in_description',
    'city_in_description',
    'parent_category_name_in_description',
    'category_name_in_description',
    'title_in_description',
])

### Save clean text features

In [45]:
text_feature = all_df['region'] + ' ' + all_df['city'] + ' ' + \
                all_df['param_1'] + ' ' + all_df['param_2'] + ' ' + all_df['param_3'] + ' ' + \
                all_df['parent_category_name'] + ' ' + all_df['category_name'] + ' ' + \
                all_df['title'] + ' ' + all_df['description']

In [46]:
stopwords = ['а', 'е', 'и', 'ж', 'м', 'о', 'на', 'не', 'ни', 'об', 'но', 'он', 'мне', 'мои', 'мож', 'она', 'они', 'оно', 'мной', 'много', 'многочисленное', 'многочисленная', 'многочисленные', 'многочисленный', 'мною', 'мой', 'мог', 'могут', 'можно', 'может', 'можхо', 'мор', 'моя', 'моё', 'мочь', 'над', 'нее', 'оба', 'нам', 'нем', 'нами', 'ними', 'мимо', 'немного', 'одной', 'одного', 'менее', 'однажды', 'однако', 'меня', 'нему', 'меньше', 'ней', 'наверху', 'него', 'ниже', 'мало', 'надо', 'один', 'одиннадцать', 'одиннадцатый', 'назад', 'наиболее', 'недавно', 'миллионов', 'недалеко', 'между', 'низко', 'меля', 'нельзя', 'нибудь', 'непрерывно', 'наконец', 'никогда', 'никуда', 'нас', 'наш', 'нет', 'нею', 'неё', 'них', 'мира', 'наша', 'наше', 'наши', 'ничего', 'начала', 'нередко', 'несколько', 'обычно', 'опять', 'около', 'мы', 'ну', 'нх', 'от', 'отовсюду', 'особенно', 'нужно', 'очень', 'отсюда', 'в', 'во', 'вон', 'вниз', 'внизу', 'вокруг', 'вот', 'восемнадцать', 'восемнадцатый', 'восемь', 'восьмой', 'вверх', 'вам', 'вами', 'важное', 'важная', 'важные', 'важный', 'вдали', 'везде', 'ведь', 'вас', 'ваш', 'ваша', 'ваше', 'ваши', 'впрочем', 'весь', 'вдруг', 'вы', 'все', 'второй', 'всем', 'всеми', 'времени', 'время', 'всему', 'всего', 'всегда', 'всех', 'всею', 'всю', 'вся', 'всё', 'всюду', 'г', 'год', 'говорил', 'говорит', 'года', 'году', 'где', 'да', 'ее', 'за', 'из', 'ли', 'же', 'им', 'до', 'по', 'ими', 'под', 'иногда', 'довольно', 'именно', 'долго', 'позже', 'более', 'должно', 'пожалуйста', 'значит', 'иметь', 'больше', 'пока', 'ему', 'имя', 'пор', 'пора', 'потом', 'потому', 'после', 'почему', 'почти', 'посреди', 'ей', 'два', 'две', 'двенадцать', 'двенадцатый', 'двадцать', 'двадцатый', 'двух', 'его', 'дел', 'или', 'без', 'день', 'занят', 'занята', 'занято', 'заняты', 'действительно', 'давно', 'девятнадцать', 'девятнадцатый', 'девять', 'девятый', 'даже', 'алло', 'жизнь', 'далеко', 'близко', 'здесь', 'дальше', 'для', 'лет', 'зато', 'даром', 'первый', 'перед', 'затем', 'зачем', 'лишь', 'десять', 'десятый', 'ею', 'её', 'их', 'бы', 'еще', 'при', 'был', 'про', 'процентов', 'против', 'просто', 'бывает', 'бывь', 'если', 'люди', 'была', 'были', 'было', 'будем', 'будет', 'будете', 'будешь', 'прекрасно', 'буду', 'будь', 'будто', 'будут', 'ещё', 'пятнадцать', 'пятнадцатый', 'друго', 'другое', 'другой', 'другие', 'другая', 'других', 'есть', 'пять', 'быть', 'лучше', 'пятый', 'к', 'ком', 'конечно', 'кому', 'кого', 'когда', 'которой', 'которого', 'которая', 'которые', 'который', 'которых', 'кем', 'каждое', 'каждая', 'каждые', 'каждый', 'кажется', 'как', 'какой', 'какая', 'кто', 'кроме', 'куда', 'кругом', 'с', 'т', 'у', 'я', 'та', 'те', 'уж', 'со', 'то', 'том', 'снова', 'тому', 'совсем', 'того', 'тогда', 'тоже', 'собой', 'тобой', 'собою', 'тобою', 'сначала', 'только', 'уметь', 'тот', 'тою', 'хорошо', 'хотеть', 'хочешь', 'хоть', 'хотя', 'свое', 'свои', 'твой', 'своей', 'своего', 'своих', 'свою', 'твоя', 'твоё', 'раз', 'уже', 'сам', 'там', 'тем', 'чем', 'сама', 'сами', 'теми', 'само', 'рано', 'самом', 'самому', 'самой', 'самого', 'семнадцать', 'семнадцатый', 'самим', 'самими', 'самих', 'саму', 'семь', 'чему', 'раньше', 'сейчас', 'чего', 'сегодня', 'себе', 'тебе', 'сеаой', 'человек', 'разве', 'теперь', 'себя', 'тебя', 'седьмой', 'спасибо', 'слишком', 'так', 'такое', 'такой', 'такие', 'также', 'такая', 'сих', 'тех', 'чаще', 'четвертый', 'через', 'часто', 'шестой', 'шестнадцать', 'шестнадцатый', 'шесть', 'четыре', 'четырнадцать', 'четырнадцатый', 'сколько', 'сказал', 'сказала', 'сказать', 'ту', 'ты', 'три', 'эта', 'эти', 'что', 'это', 'чтоб', 'этом', 'этому', 'этой', 'этого', 'чтобы', 'этот', 'стал', 'туда', 'этим', 'этими', 'рядом', 'тринадцать', 'тринадцатый', 'этих', 'третий', 'тут', 'эту', 'суть', 'чуть', 'тысяч']

In [47]:
def clean_text(txt):
    words = str(txt).split(" .!?^+-*/@~:;/\\\"\'&{}[]()#$%") #str(txt).split(" ") #
    words = [wrd for wrd in words if wrd not in stopwords and len(wrd) > 1]
    txt = " ".join(words)
    return txt

In [48]:
text_feature = text_feature.apply(lambda x: clean_text(x))

In [49]:
import pickle
with open('text_feature.pickle', 'wb') as handle:
    pickle.dump(text_feature, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [50]:
del text_feature; gc.collect()

277

In [51]:
all_df.drop(['user_id', 'title', 'description'], axis=1, inplace=True); gc.collect()

0

## Categorical Features

In [52]:
from sklearn.preprocessing import LabelEncoder as LE

In [53]:
labelize_cols = ['region', 'city', 'parent_category_name', 'category_name',
                 'param_1', 'param_2', 'param_3', 'item_seq_number', 'user_type',
                 'image_top_1']

In [54]:
MAX_CAT_TH = 3000

In [55]:
def seperate_cat_cols_by_card(cols, th=MAX_CAT_TH):
    high_card_cat_cols = []
    cat_cols = []
    for col in labelize_cols:
        feature_types = all_df[col].value_counts().index
        print('{} total types # = {}'.format(col, len(feature_types)))
        if len(feature_types) >= th:
            high_card_cat_cols.append(col)
        else:
            cat_cols.append(col)
            
    return high_card_cat_cols, cat_cols

In [56]:
def get_filtered_types(feature):
    vc = all_df[feature].value_counts()
    feature_types = vc.index
    train_len = len(train_y)
    train_filter = feature_types.isin(all_df.loc[:train_len-1, feature].values)
    test_filter = feature_types.isin(all_df.loc[train_len:, feature].values)
    final_filter = train_filter & test_filter
    
    print(feature, ': Original feature types # = {}, new feature types # = {}'.format(
            len(feature_types), pd.Series(final_filter).value_counts()[1]))
    return feature_types[final_filter]

In [57]:
high_card_cat_cols, cat_cols = seperate_cat_cols_by_card(labelize_cols)
high_card_cat_cols, cat_cols

region total types # = 28
city total types # = 1752
parent_category_name total types # = 9
category_name total types # = 47
param_1 total types # = 372
param_2 total types # = 278
param_3 total types # = 1277
item_seq_number total types # = 33947
user_type total types # = 3
image_top_1 total types # = 3064


(['item_seq_number', 'image_top_1'],
 ['region',
  'city',
  'parent_category_name',
  'category_name',
  'param_1',
  'param_2',
  'param_3',
  'user_type'])

In [58]:
for col in cat_cols:
    remain_feature_types = get_filtered_types(col)
    if type(all_df[col][0]) == str:
        all_df.loc[~all_df[col].isin(remain_feature_types), col] = 'others'
    else:
        all_df.loc[~all_df[col].isin(remain_feature_types), col] = -1
        
    all_df.loc[:, col] = LE().fit_transform(all_df[col]) 

region : Original feature types # = 28, new feature types # = 28
city : Original feature types # = 1752, new feature types # = 1625
parent_category_name : Original feature types # = 9, new feature types # = 9
category_name : Original feature types # = 47, new feature types # = 47
param_1 : Original feature types # = 372, new feature types # = 362
param_2 : Original feature types # = 278, new feature types # = 250
param_3 : Original feature types # = 1277, new feature types # = 917
user_type : Original feature types # = 3, new feature types # = 3


In [59]:
for col in high_card_cat_cols: # image_top_1, item_seq_number
    mapping = all_df.loc[:, col].value_counts()
    #mapping = mapping/mapping.sum()
    #print(mapping)
    all_df.loc[:, col+'_cnt'] = all_df[col].map(mapping)
    col_info['num_cols'].extend([col+'_cnt'])

In [60]:
col_info['high_card_cat_cols'].extend(high_card_cat_cols)
col_info['cat_cols'].extend(cat_cols)
col_info['num_cols'].extend(['item_seq_number'])

### Add Categories Interactions

In [61]:
pairs = [('region', 'city'), ('parent_category_name', 'category_name'), 
         ('parent_category_name', 'param_1'), ('parent_category_name', 'param_2'), ('parent_category_name', 'param_3'), 
         ('category_name', 'param_1'), ('category_name', 'param_2'), ('category_name', 'param_3'),
         ('parent_category_name', 'region'), ('category_name', 'region'),
         ('user_type', 'region'), ('user_type', 'city'),
         ('user_type', 'parent_category_name'), ('user_type', 'category_name'),
         #('parent_category_name', 'image_top_1'), ('category_name', 'image_top_1'),
        ]

In [62]:
for pair in pairs:
    col1 = pair[0]
    col2 = pair[1]
    print('Processing pair:', pair)
    all_df.loc[:, col1+'_'+col2] = all_df[col1].astype(str)+'_'+all_df[col2].astype(str)
    all_df.loc[:, col1+'_'+col2] = LE().fit_transform(all_df[col1+'_'+col2]) 

Processing pair: ('region', 'city')
Processing pair: ('parent_category_name', 'category_name')
Processing pair: ('parent_category_name', 'param_1')
Processing pair: ('parent_category_name', 'param_2')
Processing pair: ('parent_category_name', 'param_3')
Processing pair: ('category_name', 'param_1')
Processing pair: ('category_name', 'param_2')
Processing pair: ('category_name', 'param_3')
Processing pair: ('parent_category_name', 'region')
Processing pair: ('category_name', 'region')
Processing pair: ('user_type', 'region')
Processing pair: ('user_type', 'city')
Processing pair: ('user_type', 'parent_category_name')
Processing pair: ('user_type', 'category_name')


In [63]:
labelize_cols = ['region_city', 'parent_category_name_category_name', 'parent_category_name_param_1', 
                 'parent_category_name_param_2', 'parent_category_name_param_3', 'category_name_param_1',
                 'category_name_param_2', 'category_name_param_3', 'parent_category_name_region', 'category_name_region',
                 'user_type_region', 'user_type_city', 'user_type_parent_category_name', 'user_type_category_name']

In [64]:
high_card_cat_cols, cat_cols = seperate_cat_cols_by_card(labelize_cols)
high_card_cat_cols, cat_cols

region_city total types # = 1716
parent_category_name_category_name total types # = 47
parent_category_name_param_1 total types # = 378
parent_category_name_param_2 total types # = 263
parent_category_name_param_3 total types # = 935
category_name_param_1 total types # = 447
category_name_param_2 total types # = 315
category_name_param_3 total types # = 990
parent_category_name_region total types # = 252
category_name_region total types # = 1315
user_type_region total types # = 84
user_type_city total types # = 3806
user_type_parent_category_name total types # = 27
user_type_category_name total types # = 141


(['user_type_city'],
 ['region_city',
  'parent_category_name_category_name',
  'parent_category_name_param_1',
  'parent_category_name_param_2',
  'parent_category_name_param_3',
  'category_name_param_1',
  'category_name_param_2',
  'category_name_param_3',
  'parent_category_name_region',
  'category_name_region',
  'user_type_region',
  'user_type_parent_category_name',
  'user_type_category_name'])

In [65]:
for col in cat_cols:
    remain_feature_types = get_filtered_types(col)
    if type(all_df[col][0]) == str:
        all_df.loc[~all_df[col].isin(remain_feature_types), col] = 'others'
    else:
        all_df.loc[~all_df[col].isin(remain_feature_types), col] = -1
        
    all_df.loc[:, col] = LE().fit_transform(all_df[col]) 

region_city : Original feature types # = 1716, new feature types # = 1694
parent_category_name_category_name : Original feature types # = 47, new feature types # = 47
parent_category_name_param_1 : Original feature types # = 378, new feature types # = 376
parent_category_name_param_2 : Original feature types # = 263, new feature types # = 261
parent_category_name_param_3 : Original feature types # = 935, new feature types # = 934
category_name_param_1 : Original feature types # = 447, new feature types # = 442
category_name_param_2 : Original feature types # = 315, new feature types # = 313
category_name_param_3 : Original feature types # = 990, new feature types # = 989
parent_category_name_region : Original feature types # = 252, new feature types # = 252
category_name_region : Original feature types # = 1315, new feature types # = 1301
user_type_region : Original feature types # = 84, new feature types # = 84
user_type_parent_category_name : Original feature types # = 27, new featur

In [66]:
for col in high_card_cat_cols: #user_type_city
    mapping = all_df.loc[:, col].value_counts()
    #mapping = mapping/mapping.sum()
    #print(mapping)
    all_df.loc[:, col+'_cnt'] = all_df[col].map(mapping)
    col_info['num_cols'].extend([col+'_cnt'])

In [67]:
col_info['high_card_cat_cols'].extend(high_card_cat_cols)
col_info['cat_cols'].extend(cat_cols)

In [68]:
# city contains repeated names
all_df.drop(['city'], axis=1, inplace=True); gc.collect()
col_info['cat_cols'].remove('city')

## Contiguos Features: Price

In [69]:
col_info['num_cols'].extend(['price'])

In [70]:
col_info

{'cat_cols': ['region',
  'parent_category_name',
  'category_name',
  'param_1',
  'param_2',
  'param_3',
  'user_type',
  'region_city',
  'parent_category_name_category_name',
  'parent_category_name_param_1',
  'parent_category_name_param_2',
  'parent_category_name_param_3',
  'category_name_param_1',
  'category_name_param_2',
  'category_name_param_3',
  'parent_category_name_region',
  'category_name_region',
  'user_type_region',
  'user_type_parent_category_name',
  'user_type_category_name'],
 'high_card_cat_cols': ['item_seq_number', 'image_top_1', 'user_type_city'],
 'num_cols': ['same_activate_cnt',
  'duration_1_mean',
  'duration_1_med',
  'duration_1_min',
  'duration_1_max',
  'duration_1_std',
  'duration_2_mean',
  'duration_2_med',
  'duration_2_min',
  'duration_2_max',
  'duration_2_std',
  'duration_3_mean',
  'duration_3_med',
  'duration_3_min',
  'duration_3_max',
  'duration_3_std',
  'renewed_count_mean',
  'renewed_count_med',
  'renewed_count_min',
  're

## Dates Features ?

In [71]:
all_df["Weekday"] = all_df['activation_date'].dt.weekday
#all_df["woy"] = all_df['activation_date'].dt.week
all_df["dom"] = all_df['activation_date'].dt.day

In [72]:
labelize_cols = ["Weekday", 'dom']
for col in labelize_cols:
    all_df.loc[:, col] = LE().fit_transform(all_df[col]) 

In [73]:
all_df.drop(['activation_date'], axis=1, inplace=True)

In [74]:
all_df.head(3)

,item_id,region,parent_category_name,category_name,param_1,param_2,param_3,price,item_seq_number,user_type,image_top_1,deal_probability,same_activate_cnt,duration_1_mean,duration_1_med,duration_1_min,duration_1_max,duration_1_std,duration_2_mean,duration_2_med,duration_2_min,duration_2_max,duration_2_std,duration_3_mean,duration_3_med,duration_3_min,duration_3_max,duration_3_std,renewed_count_mean,renewed_count_med,renewed_count_min,renewed_count_max,renewed_count_std,is_renewed_mean,is_renewed_med,is_renewed_std,avg_days_up_user,avg_times_up_user,med_days_up_user,med_times_up_user,n_user_items,price_mean,price_med,price_std,title_len_mean,title_len_med,title_len_std,title_word_cnt_mean,title_word_cnt_med,title_word_cnt_std,...,image_top_1_is_na,region_in_title,region_in_title_counts,city_in_title,city_in_title_counts,parent_category_name_in_title,parent_category_name_in_title_counts,category_name_in_title,category_name_in_title_counts,region_in_description,region_in_description_counts,city_in_description,city_in_description_counts,parent_category_name_in_description,parent_category_name_in_description_counts,category_name_in_description,category_name_in_description_counts,title_in_description,title_in_description_counts,desc_char_count,desc_space_count,desc_unique_words_count,desc_percnt_unique_words,surprise_count,question_count,quote_count,quote_count2,tit_char_count,tit_space_count,tit_unique_words_count,tit_percnt_unique_words,item_seq_number_cnt,image_top_1_cnt,region_city,parent_category_name_category_name,parent_category_name_param_1,parent_category_name_param_2,parent_category_name_param_3,category_name_param_1,category_name_param_2,category_name_param_3,parent_category_name_region,category_name_region,user_type_region,user_type_city,user_type_parent_category_name,user_type_category_name,user_type_city_cnt,Weekday,dom
0,3b896605f03c,17,4,10,109,103,156,500.0,12,1,1008.0,0.0,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023,6.5,1.5,6.5,1.5,5.0,0.690736,0.450292,0.875487,0.8649,0.842954,0.997657,0.864585,0.771784,0.966502,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,28,3,4,100.0,0,0,0,0,17,1,2,100.0,31778,680,511,23,238,49,28,5,91,840,121,38,37,1641,13,49,39989,2,7
1,c81c34d3aac5,6,0,41,41,241,875,100.0,5,1,2909.0,0.0,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023,15.0,2.0,15.0,2.0,2.0,0.690736,0.450292,0.875487,0.8649,0.842954,0.997657,0.864585,0.771784,0.966502,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,1,2,100.0,0,0,0,0,19,3,4,100.0,50246,6126,1391,6,65,20,1,355,268,971,24,990,52,2160,9,83,5595,2,7
2,a8ab6225dd06,15,4,29,125,103,121,1500.0,32,1,527.0,0.0,716790.0,12.912948,0.0,0.0,33.0,13.951994,20.236426,13.0,0.0,34.0,9.02955,7.323478,6.0,0.0,13.0,5.555416,0.445527,0.0,0.0,2.0,0.497047,0.445515,0.0,0.497023,0.0,0.0,0.0,0.0,0.0,0.690736,0.450292,0.875487,0.8649,0.842954,0.997657,0.864585,0.771784,0.966502,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,1,100.0,0,0,0,0,22,2,3,100.0,14923,5321,402,25,239,49,16,153,156,924,119,582,35,2670,13,69,1093,2,7


In [75]:
col_info['cat_cols'].extend(labelize_cols)

## Special Encodings

### Average prices for each high card categories

In [76]:
for col in ['image_top_1', 'item_seq_number', 'user_type_city']:
    mapping = all_df.groupby(col)['price'].apply(lambda x: x[x>=0].mean())
    mapping_std = all_df.groupby(col)['price'].apply(lambda x: x[x>=0].std())
    
    all_df.loc[:,col + '_mean_price'] = all_df[col].map(mapping)
    all_df.loc[:,col + '_std_price'] = all_df[col].map(mapping_std)
    print(col + '_mean\std_price processed')
    
    del mapping; gc.collect()

encoded_cols = ['image_top_1_mean_price', 'image_top_1_std_price', 
                'item_seq_number_mean_price', 'item_seq_number_std_price',
                'user_type_city_mean_price', 'user_type_city_std_price', ]
all_df.loc[:,encoded_cols] = all_df[encoded_cols].fillna(0.)

image_top_1_mean\std_price processed
item_seq_number_mean\std_price processed
user_type_city_mean\std_price processed


In [77]:
all_df.loc[:5, encoded_cols]

,image_top_1_mean_price,image_top_1_std_price,item_seq_number_mean_price,item_seq_number_std_price,user_type_city_mean_price,user_type_city_std_price
0,20839.372197,483202.915713,158210.429546,3.089577e+06,131776.415479,2.003123e+06
1,12748.863477,840848.174566,211505.927010,2.100275e+06,76308.339231,8.161533e+05
2,7727.267606,297440.003819,113293.395794,1.124093e+06,98559.301868,4.217653e+05
3,472.655319,393.283928,169508.827833,3.079037e+06,78159.809318,6.206042e+05
4,2025.829690,2501.143785,194301.370019,2.908802e+06,64769.557860,8.088853e+05
5,1537.640335,7774.390175,244022.223536,5.626242e+06,138628.842312,1.114809e+06


In [78]:
col_info['num_cols'].extend(encoded_cols)

### OOF mean enc of deal probability for each type of price, item_seq_number (in 200 quantiles), image_top1

In [79]:
all_df.shape

(2011862, 133)

In [80]:
all_df.loc[:, 'price_binned'] = pd.qcut(all_df.price, 200, labels=False, duplicates='drop', retbins=False)

In [81]:
from sklearn.model_selection import KFold
fold_num = 10
kf = KFold(fold_num, shuffle=True, random_state=411519719)

mean_enc_cols = ['price_binned', 'item_seq_number', 'image_top_1', 'user_type_city']
train_df = cp(all_df.loc[:len(train_y)-1, mean_enc_cols])
train_df.loc[:, 'deal_probability'] = train_y

for col in mean_enc_cols:    
    
    all_df.loc[:, col+'_dp_mean_enc'] = np.zeros((all_df.shape[0],))
    all_df.loc[:, col+'_dp_std_enc'] = np.zeros((all_df.shape[0],))
    
    for train_ix, val_ix in kf.split(train_df):
        tr_X = train_df.loc[train_ix, :]
    
        gp = tr_X.groupby(col)['deal_probability']
        mapping, mapping_std = gp.mean(), gp.std()
        
        all_df.loc[val_ix, col+'_dp_mean_enc'] = all_df.loc[val_ix, col].map(mapping)
        all_df.loc[val_ix, col+'_dp_std_enc'] = all_df.loc[val_ix, col].map(mapping_std)
        
        all_df.loc[len(train_y):, col+'_dp_mean_enc'] += all_df.loc[len(train_y):, col].map(mapping)
        all_df.loc[len(train_y):, col+'_dp_std_enc'] += all_df.loc[len(train_y):, col].map(mapping_std)
     
    all_df.loc[len(train_y):, col+'_dp_mean_enc'] /= fold_num
    all_df.loc[len(train_y):, col+'_dp_std_enc'] /= fold_num
    print(col + ' processed.')
    
encoded_cols = ['price_binned_dp_mean_enc', 'item_seq_number_dp_mean_enc', 'image_top_1_dp_mean_enc', 
                'user_type_city_dp_mean_enc',
                'price_binned_dp_std_enc', 'item_seq_number_dp_std_enc', 'image_top_1_dp_std_enc',
                'user_type_city_dp_std_enc']

all_df.loc[:,encoded_cols] = all_df[encoded_cols].fillna(0.)
del train_df; gc.collect()

price_binned processed.
item_seq_number processed.
image_top_1 processed.
user_type_city processed.


346

In [82]:
all_df.loc[:5, encoded_cols]

,price_binned_dp_mean_enc,item_seq_number_dp_mean_enc,image_top_1_dp_mean_enc,user_type_city_dp_mean_enc,price_binned_dp_std_enc,item_seq_number_dp_std_enc,image_top_1_dp_std_enc,user_type_city_dp_std_enc
0,0.104026,0.136740,0.081370,0.136078,0.305296,0.343580,0.273696,0.342877
1,0.102374,0.134580,0.033991,0.129346,0.303144,0.341280,0.181227,0.335625
2,0.131337,0.121455,0.040822,0.105702,0.337772,0.326671,0.197903,0.307670
3,0.084343,0.133014,0.126506,0.135701,0.277908,0.339600,0.332921,0.342483
4,0.132263,0.137637,0.075227,0.126933,0.338780,0.344524,0.263928,0.332908
5,0.093142,0.136781,0.022493,0.126016,0.290637,0.343621,0.148303,0.331878


In [83]:
col_info['other_cols'].extend(encoded_cols)
col_info['cat_cols'].extend(['price_binned'])
col_info

{'cat_cols': ['region',
  'parent_category_name',
  'category_name',
  'param_1',
  'param_2',
  'param_3',
  'user_type',
  'region_city',
  'parent_category_name_category_name',
  'parent_category_name_param_1',
  'parent_category_name_param_2',
  'parent_category_name_param_3',
  'category_name_param_1',
  'category_name_param_2',
  'category_name_param_3',
  'parent_category_name_region',
  'category_name_region',
  'user_type_region',
  'user_type_parent_category_name',
  'user_type_category_name',
  'Weekday',
  'dom',
  'price_binned'],
 'high_card_cat_cols': ['item_seq_number', 'image_top_1', 'user_type_city'],
 'num_cols': ['same_activate_cnt',
  'duration_1_mean',
  'duration_1_med',
  'duration_1_min',
  'duration_1_max',
  'duration_1_std',
  'duration_2_mean',
  'duration_2_med',
  'duration_2_min',
  'duration_2_max',
  'duration_2_std',
  'duration_3_mean',
  'duration_3_med',
  'duration_3_min',
  'duration_3_max',
  'duration_3_std',
  'renewed_count_mean',
  'renewed_

In [84]:
all_df.drop(['deal_probability', 'item_id'], axis=1, inplace=True)

## Boxcox + Normalize columns

In [85]:
from scipy.stats import boxcox
for c in col_info['num_cols']:
    val = all_df.loc[all_df[c]>=0, c]
    
    if len(val.unique()) == 1:
        all_df.drop([c], axis=1, inplace=True)
        continue
  
    val = (val-val.min())/(val.max()-val.min())
    val, ld = boxcox(val+1e-15)
    all_df.loc[all_df[c]>=0, c] = (val-val.min())/(val.max()-val.min())
    
    print('Column {} transformed with optimal lambda value={}'.format(c, ld))
    del val; gc.collect()

Column same_activate_cnt transformed with optimal lambda value=2.7114969960996764
Column duration_1_mean transformed with optimal lambda value=0.24394822407932282
Column duration_1_med transformed with optimal lambda value=-0.5155508596614008


C:\Users\khyeh\Anaconda3\lib\site-packages\scipy\stats\morestats.py:901: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\khyeh\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1927: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


Column duration_1_min transformed with optimal lambda value=-10.34280112063192
Column duration_1_max transformed with optimal lambda value=0.34016332193804744
Column duration_1_std transformed with optimal lambda value=0.2640064783483801
Column duration_2_mean transformed with optimal lambda value=0.36515781488628557
Column duration_2_med transformed with optimal lambda value=0.3008570092076287
Column duration_2_min transformed with optimal lambda value=-10.34280112063192
Column duration_2_max transformed with optimal lambda value=0.33974572758479105
Column duration_2_std transformed with optimal lambda value=0.3040210536067638
Column duration_3_mean transformed with optimal lambda value=0.43242013809069346
Column duration_3_med transformed with optimal lambda value=0.3973204981127148
Column duration_3_max transformed with optimal lambda value=0.39897141216527005
Column duration_3_std transformed with optimal lambda value=0.28778006644105375
Column renewed_count_mean transformed with o

In [86]:
na_sum = all_df.isnull().sum()
na_sum[na_sum>0]

Series([], dtype: int64)

In [87]:
all_df.isnull().sum().max()

0

In [88]:
col_info

{'cat_cols': ['region',
  'parent_category_name',
  'category_name',
  'param_1',
  'param_2',
  'param_3',
  'user_type',
  'region_city',
  'parent_category_name_category_name',
  'parent_category_name_param_1',
  'parent_category_name_param_2',
  'parent_category_name_param_3',
  'category_name_param_1',
  'category_name_param_2',
  'category_name_param_3',
  'parent_category_name_region',
  'category_name_region',
  'user_type_region',
  'user_type_parent_category_name',
  'user_type_category_name',
  'Weekday',
  'dom',
  'price_binned'],
 'high_card_cat_cols': ['item_seq_number', 'image_top_1', 'user_type_city'],
 'num_cols': ['same_activate_cnt',
  'duration_1_mean',
  'duration_1_med',
  'duration_1_min',
  'duration_1_max',
  'duration_1_std',
  'duration_2_mean',
  'duration_2_med',
  'duration_2_min',
  'duration_2_max',
  'duration_2_std',
  'duration_3_mean',
  'duration_3_med',
  'duration_3_min',
  'duration_3_max',
  'duration_3_std',
  'renewed_count_mean',
  'renewed_

In [89]:
drop_cols = [col for col in col_info['high_card_cat_cols'] if col not in col_info['num_cols']]
all_df.drop(drop_cols, axis=1, inplace=True)

## Transform into sparse matrix and Save!

In [90]:
with open('all_features_dense.pickle', 'wb') as handle:
    pickle.dump(all_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [91]:
column_filt = all_df.columns.isin(col_info['cat_cols'])
pd.Series(column_filt).value_counts()

False    113
True      23
dtype: int64

In [92]:
all_df.columns.tolist()

['region',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'price',
 'item_seq_number',
 'user_type',
 'same_activate_cnt',
 'duration_1_mean',
 'duration_1_med',
 'duration_1_min',
 'duration_1_max',
 'duration_1_std',
 'duration_2_mean',
 'duration_2_med',
 'duration_2_min',
 'duration_2_max',
 'duration_2_std',
 'duration_3_mean',
 'duration_3_med',
 'duration_3_max',
 'duration_3_std',
 'renewed_count_mean',
 'renewed_count_med',
 'renewed_count_max',
 'renewed_count_std',
 'is_renewed_mean',
 'is_renewed_med',
 'is_renewed_std',
 'avg_days_up_user',
 'avg_times_up_user',
 'med_days_up_user',
 'med_times_up_user',
 'n_user_items',
 'price_mean',
 'price_med',
 'price_std',
 'title_len_mean',
 'title_len_med',
 'title_len_std',
 'title_word_cnt_mean',
 'title_word_cnt_med',
 'title_word_cnt_std',
 'title_unique_word_cnt_mean',
 'title_unique_word_cnt_med',
 'title_unique_word_cnt_std',
 'title_unqiue_percent_mean',
 'title_unqiue_percent_std',
 'descr

In [93]:
from sklearn.preprocessing import OneHotEncoder
ohe_features = OneHotEncoder(categorical_features=column_filt).fit_transform(all_df)

In [94]:
del all_df; gc.collect()

130

In [95]:
from scipy.sparse import csr_matrix
ohe_features = csr_matrix(ohe_features)

In [96]:
with open('all_features.pickle', 'wb') as handle:
    pickle.dump(ohe_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [97]:
print(type(ohe_features))
ohe_features.shape

<class 'scipy.sparse.csr.csr_matrix'>


(2011862, 8735)